## Prob 1



###시도 1: VGG16 
수업시간에 했던 모델 중 VGG16이 가장 높아 VGG16으로 선택했다.<br>
또한, num_epoch를 50에서 60으로 올리면 더 모델 성능이 더 올라갈 것 같아 60으로 변경해서 돌려봤다.




VGG16 모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 합성곱 연산 2번
def conv_2_block(in_dim, out_dim): 
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding =1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

# 합성곱 연산 3번
def conv_3_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

In [ ]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64, 128),
        conv_3_block(128, 256),
        conv_3_block(256, 512),
        conv_3_block(512, 512)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512, 100),
        nn.ReLU(True),
        nn.Linear(100, 20),
        nn.ReLU(True),
        nn.Linear(20, 10),
    )
  
  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

In [ ]:
#batch사이즈, 학습률, num_epoch 정의 
batch_size = 32 
learning_rate = 0.0001
num_epoch = 60    #num_epoch를 60으로!

CIFAR10 데이터셋 불러오기

In [ ]:
transf = transforms.Compose([
                             transforms.ToTensor(),
])

In [ ]:
trainset = dset.CIFAR10(root = './data', train=True, download=True, transform=transf)
testset = dset.CIFAR10(root='./data', train=False, download = True, transform=transf)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)

GPU연산 체크

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = VGG().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

cuda:0


학습하기

In [ ]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

  if i % 10 == 0 :
    print(loss)

tensor(1.9949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3525, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1867, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0527, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0113, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0172, device='cuda:0', grad_fn=<NllLossBackward0>)


성능 평가하기

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 77.92 %


###시도 2 : VGG16 두번째 부분을 합성곱연산 3번으로 변경
num_epoch는 50보다 60이 더 크므로 유지하고, VGG16에서 두번째 부분을 합성곱 연산을 2번이 아닌 3번으로 했을 때 성능이 더 높아질 거 같아 변경시켜봤다.


모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#합성곱 연산 2번
def conv2_2_block(in_dim, out_dim):
  model2 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding =1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model2

#합성곱 연산 3번
def conv2_3_block(in_dim, out_dim):
  model2 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model2

In [ ]:
class VGG2(nn.Module):
  def __init__(self):
    super(VGG2, self).__init__()
    self.feature = nn.Sequential(
        conv2_2_block(3,64),
        conv2_3_block(64, 128),  # 합성곱 연산 2번이 아닌 3번으로 변경
        conv2_3_block(128, 256),
        conv2_3_block(256, 512),
        conv2_3_block(512, 512)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512, 100),
        nn.ReLU(True),
        nn.Linear(100, 20),
        nn.ReLU(True),
        nn.Linear(20, 10),
    )
  
  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

CIFAR10 데이터셋 불러오기

In [ ]:
#batch사이즈, 학습률, num_epoch 정의 
batch_size = 32
learning_rate = 0.0001
num_epoch = 60

transf = transforms.Compose([
                             transforms.ToTensor(),
])

trainset = dset.CIFAR10(root = './data', train=True, download=True, transform=transf)
testset = dset.CIFAR10(root='./data', train=False, download = True, transform=transf)

trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)

Files already downloaded and verified
Files already downloaded and verified


GPU연산 체크

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model2 = VGG2().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr = learning_rate)

cuda:0


학습하기

In [ ]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model2.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

  if i % 10 == 0 :
    print(loss)

tensor(2.2625, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0152, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0036, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0013, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0212, device='cuda:0', grad_fn=<NllLossBackward0>)


성능 평가하기

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model2.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model2(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 80.60 %


###시도 3 : VGG16에서 모든 부분 합성곱연산 3번으로 변경
시도2에서 성능이 더 높아져 첫번째 부분까지 모두 합성곱 연산을 3번으로 하면 성능이 더 높아질 것 같다는 생각에 모든 부분을 합성곱 연산 3번으로 변경해봤다.

모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

def conv3_3_block(in_dim, out_dim):
  model3 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model3

In [ ]:
class VGG3(nn.Module):
  def __init__(self):
    super(VGG3, self).__init__()
    self.feature = nn.Sequential(
        conv3_3_block(3,64),      # 합성곱 연산 2번이 아닌 3번으로 변경
        conv3_3_block(64, 128),
        conv3_3_block(128, 256),
        conv3_3_block(256, 512),
        conv3_3_block(512, 512)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(512, 100),
        nn.ReLU(True),
        nn.Linear(100, 20),
        nn.ReLU(True),
        nn.Linear(20, 10),
    )
  
  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

CIFAR10 데이터셋 불러오기

In [ ]:
batch_size = 32
learning_rate = 0.0001
num_epoch = 60

transf = transforms.Compose([
                             transforms.ToTensor(),
])

trainset = dset.CIFAR10(root = './data', train=True, download=True, transform=transf)
testset = dset.CIFAR10(root='./data', train=False, download = True, transform=transf)

trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)

Files already downloaded and verified
Files already downloaded and verified


GPU연산 체크

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model3 = VGG3().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr = learning_rate)

cuda:0


학습하기

In [ ]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model3.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

  if i % 10 == 0 :
    print(loss)

tensor(2.3174, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1131, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1996, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0281, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0123, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0010, device='cuda:0', grad_fn=<NllLossBackward0>)


성능 평가하기

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model3.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model3(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 72.90 %


 ### 최종 모델을 파일로 저장

최종 성능 **80.6%**로 model2(VGG16 두번째 부분 합성곱 연산 3번으로 변경)가 가장 높은 성능을 기록한다. 

In [ ]:
#model2 저장
torch.save(model2, 'model2.pt')

In [ ]:
model2 = torch.load('model2.pt')

In [ ]:
#저장한 model2 출력해보기
model2

VGG2(
  (feature): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(256, 256, kernel_size=(3, 3), s

## Prob 2 

###시도1: VGG16
여러가지 모델들을 시도해봤지만, 가장 높은 VGG16으로 선택했다.

VGG16 모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


def conv_2_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding =1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

def conv_3_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

In [ ]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3,64),
        conv_2_block(64, 128),
        conv_3_block(128, 256),
        conv_3_block(256, 512),
        conv_3_block(512, 512)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(4608, 100),
        nn.ReLU(True),
        nn.Linear(100, 20),
        nn.ReLU(True),
        nn.Linear(20, 10),
    )
  
  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

STL-10 데이터셋 불러오기

In [ ]:
batch_size = 32
learning_rate = 0.0001
num_epoch = 60

transf = transforms.Compose([
          transforms.ToTensor(),
])

trainset = dset.STL10(root = './data', split = 'train', download=True, transform=transf)
testset = dset.STL10(root='./data', split = 'test', download = True, transform=transf)

trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


GPU연산 체크

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = VGG().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

cuda:0


학습시키기

In [ ]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

  if i % 10 == 0 :
    print(loss)

tensor(2.3363, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3037, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2711, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1297, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0025, device='cuda:0', grad_fn=<NllLossBackward0>)


성능 평가하기

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 53.34 %


###시도 2 : VGG16 두번째 부분을 합성곱연산 3번으로 변경
prob1번에서 이 부분이 가장 높았어서 이번에도 VGG16보다 높지 않을까?라는 생각에 시도해봤다.

모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


def conv2_2_block(in_dim, out_dim):
  model2 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding =1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model2

def conv2_3_block(in_dim, out_dim):
  model2 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model2

In [ ]:
class VGG2(nn.Module):
  def __init__(self):
    super(VGG2, self).__init__()
    self.feature = nn.Sequential(
        conv2_2_block(3,64),
        conv2_3_block(64, 128),     # 합성곱 연산 2번이 아닌 3번으로 변경
        conv2_3_block(128, 256),
        conv2_3_block(256, 512),
        conv2_3_block(512, 512)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(4608, 100),
        nn.ReLU(True),
        nn.Linear(100, 20),
        nn.ReLU(True),
        nn.Linear(20, 10),
    )
  
  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

STL-10 데이터셋 불러오기

In [ ]:
batch_size = 32
learning_rate = 0.0001
num_epoch = 60

transf = transforms.Compose([
          transforms.ToTensor(),
])

trainset = dset.STL10(root = './data', split = 'train', download=True, transform=transf)
testset = dset.STL10(root='./data', split = 'test', download = True, transform=transf)

trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)

Files already downloaded and verified
Files already downloaded and verified


GPU연산 체크

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model2 = VGG2().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr = learning_rate)

cuda:0


학습하기

In [ ]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model2.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

  if i % 10 == 0 :
    print(loss)

tensor(2.3271, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3087, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9822, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1060, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8188, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3797, device='cuda:0', grad_fn=<NllLossBackward0>)


성능 평가하기

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model2.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model2(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 48.19 %


###시도 3 : VGG16의 num_epoch 70으로 변경
첫번째 시도가 가장 높아서 num_epoch를 60에서 70으로 변경하면 성능이 더 올라갈 것 같은 생각에 시도해봤다.

모델 구현하기

In [ ]:
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


def conv3_2_block(in_dim, out_dim):
  model3 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding =1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model3

def conv3_3_block(in_dim, out_dim):
  model3 = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model3

In [ ]:
class VGG3(nn.Module):
  def __init__(self):
    super(VGG3, self).__init__()
    self.feature = nn.Sequential(
        conv3_2_block(3,64),
        conv3_2_block(64, 128),
        conv3_3_block(128, 256),
        conv3_3_block(256, 512),
        conv3_3_block(512, 512)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(4608, 100),
        nn.ReLU(True),
        nn.Linear(100, 20),
        nn.ReLU(True),
        nn.Linear(20, 10),
    )
  
  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

STL-10 데이터셋 불러오기 

In [ ]:
batch_size = 32
learning_rate = 0.0001
num_epoch = 70          #60에서 70으로 변경

transf = transforms.Compose([
          transforms.ToTensor(),
])

trainset = dset.STL10(root = './data', split = 'train', download=True, transform=transf)
testset = dset.STL10(root='./data', split = 'test', download = True, transform=transf)

trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)

Files already downloaded and verified
Files already downloaded and verified


GPU 연산 체크하기

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model3 = VGG3().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr = learning_rate)

cuda:0


학습하기

In [ ]:
for i in range(num_epoch):
  for j, [image, label] in enumerate(trainloader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model3.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

  if i % 10 == 0 :
    print(loss)

tensor(2.2141, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2225, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5340, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0755, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0502, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0023, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)


성능 평가하기

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  model3.eval()
  for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model3(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Test accuracy: %.2f %%'%(100*correct/total))

Test accuracy: 49.92 %


최종 성능 **53.34%**로 모델1(VGG16)에서 가장 높은 성능을 기록한다.